In [ ]:
# to do - 1. list page - get links to products
# 2. item page - get prodcuts data, price

In [ ]:
# todo: take first two results, calculate with bm25

In [10]:
ua = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from bs4 import BeautifulSoup
import requests

response = requests.get(
    url='https://shop.by/stiralnye_mashiny/?page_id=1',
    # url='https://yandex.by/search/',
    # params={
    #     'text':'Xiaomi 14',
    #     'lr':157,
    #     'search_source':'yaby_desktop_common',
    #     'src':'suggest_B',
    # },
    # headers={'User-Agent': ua}
)

response

soup = BeautifulSoup(response.text, "html.parser")

soup.contents

['html',
 '\n',
 <html lang="ru">
 <head>
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
 <title>Стиральная машина купить в Минске недорого, цены в интернет-магазинах – Shop.by</title>
 <meta content="Купить стиральную машину (стиралку) в каталоге 🎁Shop.by. ✔️Большой выбор, %скидки%. 💲Доступные цены. 🚚 Доставка по Минску и всей Беларуси." name="description"/>
 <meta content="Стиральная машина купить в Минске недорого, цены в интернет-магазинах – Shop.by" property="og:title"/>
 <meta content="Купить стиральную машину (стиралку) в каталоге 🎁Shop.by. ✔️Большой выбор, %скидки%. 💲Доступные цены. 🚚 Доставка по Минску и всей Беларуси." property="og:description"/>
 <meta content="https://shop.by/section/644.jpg" property="og:image"/>
 <meta content="https://shop.by/stiralnye_mashiny/" property="og:url"/>
 <meta content="Торговый портал Shop.by" property="og:site_name">
 <meta content="website" property="og:type">
 <link href="https://shop.by/stiralnye_mashiny/" rel="cano

In [11]:
shop_url = 'https://shop.by/stiralnye_mashiny/'

In [12]:



class QuotesSpider(CrawlSpider):
    name = "quotes"
    
    def __init__(self, urls_file, *a, **kw):
        super(QuotesSpider, self).__init__(*a, **kw)
    def start_requests(self):
        urls = [
            # 'http://quotes.toscrape.com/page/1/',
            # 'http://quotes.toscrape.com/page/2/',
            'https://shop.by/stiralnye_mashiny/?page_id=1',
            'https://shop.by/stiralnye_mashiny/?page_id=2',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
            
    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = 'quotes-%s.html' % page
        with open(filename, 'wb') as f:
            f.write(response.body)
        self.log('Saved file %s' % filename)


from scrapy.crawler import CrawlerProcess
c = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36',
    'FEED_FORMAT': 'csv',
    'FEED_URI': 'output.csv',
    'DEPTH_LIMIT': 2,
    'CLOSESPIDER_PAGECOUNT': 3,
})
c.crawl(QuotesSpider, urls_file='input.txt')
c.start()

2024-04-05 07:20:15 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-04-05 07:20:15 [scrapy.utils.log] INFO: Versions: lxml 5.1.0.0, libxml2 2.12.3, cssselect 1.2.0, parsel 1.9.0, w3lib 2.1.2, Twisted 24.3.0, Python 3.11.8 (main, Feb 26 2024, 21:39:34) [GCC 11.2.0], pyOpenSSL 24.1.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.5, Platform Linux-6.5.0-26-generic-x86_64-with-glibc2.35
2024-04-05 07:20:15 [scrapy.addons] INFO: Enabled addons:
[]
2024-04-05 07:20:15 [py.warnings] WARNING: /home/amstel/miniconda3/envs/langchain/lib/python3.11/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documen

In [2]:
url = 'https://shop.by/stiralnye_mashiny/?page_id=1'

In [1]:
from bs4 import BeautifulSoup
from typing import Dict
from extruct.w3cmicrodata import MicrodataExtractor
from extruct.microformat import MicroformatExtractor
import pprint

def parse_product(el) -> Dict[str, str]:  
    '''return product details from microdata'''
    product_category = el.get('properties').get('category')
    product_brand = el.get('properties').get('brand')
    product_name = el.get('properties').get('name')
    product_url = el.get('properties').get('url')
    product_image_url = el.get('properties').get('image')
    _aggredagate_rating = el.get('properties').get('aggregateRating').get('properties')
    product_rating_value = _aggredagate_rating.get('ratingValue')
    product_best_rating = _aggredagate_rating.get('bestRating')
    product_worst_rating = _aggredagate_rating.get('worstRating')
    product_rating_count = _aggredagate_rating.get('ratingCount')
    product_review_count = _aggredagate_rating.get('reviewCount')
    return {
        'product_category':product_category,
        'product_brand':product_brand,
        'product_name':product_name,
        'product_url':product_url,
        'product_image_url':product_image_url,
        'product_rating_value':product_rating_value,
        'product_best_rating':product_best_rating,
        'product_worst_rating':product_worst_rating,
        'product_rating_count':product_rating_count,
        'product_review_count':product_review_count,
    }

def parse_reviews(el) -> str:
    '''returns reviews delimited by /n from microdata'''
    total_reviews = []
    _reviews = el.get('properties').get('review')
    for review in _reviews:
        review_date_published = review.get('properties').get('datePublished')
        review_description = review.get('properties').get('description')
        _review_rating_properties = review.get('properties').get('reviewRating').get('properties')
        review_ratng_value = _review_rating_properties.get('ratingValue')
        review_best_rating = _review_rating_properties.get('bestRating')
        total_reviews.append(review_description)
    return '\n\n'.join(total_reviews)

In [14]:
!ls

duck_google_yandex.ipynb  extract_item_names.py  reviews_res.html
duck_res.html		  langchain_duck.py	 scrapy_onliner
example2.png		  output.csv		 search_duck.py
example3.png		  parse_reviews.py	 summarize_reviews_step1.py
example.png		  product_res.html	 wah_spider.html


In [15]:
with open("wash_spider.html") as fp:
# with open("product_res.html") as fp:
    soup = BeautifulSoup(fp, "html.parser")

In [18]:
import extruct
import pprint

In [19]:

pp = pprint.PrettyPrinter(indent=2)
# mde = MicrodataExtractor()
data = extruct.extract(soup.prettify(), syntaxes=['microdata'])
# pp.pprint(data)

In [20]:
data.keys()

dict_keys(['microdata'])

In [21]:
for el in data['microdata']:
    if el.get('type') == 'https://schema.org/ItemList':
        _items = el['properties'].get('itemListElement')
        for item in _items:
            if item.get('type') == 'https://schema.org/ListItem':
                item_properties = item.get('properties')
                item_url = item_properties.get('url')
                item_name = item_properties.get('name')
                item_position = item_properties.get('position')
                item_description = item_properties.get('description')
                break

In [27]:
pprint.pprint(item)

{'properties': {'description': 'Отдельно стоящая, барабанного типа, глубина '
                               '41.5 см, загрузка фронтальная, 6 кг, '
                               'количество программ 15, класс '
                               'энергопотребления А++, материал бака пластик, '
                               'отложенный старт, обработка паром, индикация '
                               'ошибок, звуковой сигнал, защита от детей, '
                               'контроль дисбаланса, контроль пенообразования, '
                               'ширина 60 см.',
                'image': {'properties': {'image': '/images/a_beko_wsre6512zaa_icon.webp'},
                          'type': 'http://schema.org/ImageObject'},
                'name': 'Стиральная машина BEKO WSRE6512ZAA',
                'position': '1',
                'url': '/stiralnye_mashiny/beko_wsre6512zaa/#shop'},
 'type': 'https://schema.org/ListItem'}


In [25]:
fields = {
  "standalone": "standalone",
  "drum_type": "drum_type",
  "depth": "depth",
  "front_loading": "front_loading",
  "weight": "weight",
  "number_of_programs": "number_of_programs",
  "energy_class": "energy_class",
  "material_of_tub": "material_of_tub",
  "delayed_start": "delayed_start",
  "steam_generation": "steam_generation",
  "error_indication": "error_indication",
  "sound_signal": "sound_signal",
  "child_protection": "child_protection",
  "balance_control": "balance_control",
  "foam_control": "foam_control",
  "width": "width"
}


In [ ]:
get

In [26]:
{k:v.strip(' ') for k,v in zip(fields.keys(), item_description.split(','))}

{'standalone': 'Отдельно стоящая',
 'drum_type': 'барабанного типа',
 'depth': 'глубина 41.5 см',
 'front_loading': 'загрузка фронтальная',
 'weight': '6 кг',
 'number_of_programs': 'количество программ 15',
 'energy_class': 'класс энергопотребления А++',
 'material_of_tub': 'материал бака пластик',
 'delayed_start': 'отложенный старт',
 'steam_generation': 'обработка паром',
 'error_indication': 'индикация ошибок',
 'sound_signal': 'звуковой сигнал',
 'child_protection': 'защита от детей',
 'balance_control': 'контроль дисбаланса',
 'foam_control': 'контроль пенообразования',
 'width': 'ширина 60 см.'}

In [24]:
item_description

'Отдельно стоящая, барабанного типа, глубина 41.5 см, загрузка фронтальная, 6 кг, количество программ 15, класс энергопотребления А++, материал бака пластик, отложенный старт, обработка паром, индикация ошибок, звуковой сигнал, защита от детей, контроль дисбаланса, контроль пенообразования, ширина 60 см.'

In [28]:
i = 0
for el in data:
    if el.get('type') == 'https://schema.org/Product':
        if 'properties' in el:
            product_details = parse_product(el)
            product_reviews = parse_reviews(el)
            i+=1

In [40]:
product_details

{'product_category': 'Мобильные телефоны',
 'product_brand': 'Xiaomi',
 'product_name': 'Смартфон Xiaomi 14',
 'product_url': 'https://market.yandex.by/product--smartfon-xiaomi-14/1943226316/reviews',
 'product_image_url': 'https://avatars.mds.yandex.net/get-mpic/4818396/img_id4780156176612636343.jpeg/x332_trim',
 'product_rating_value': '5',
 'product_best_rating': '5',
 'product_worst_rating': '1',
 'product_rating_count': '45',
 'product_review_count': '20'}

In [42]:
from extruct.jsonld import JsonLdExtractor
from extruct.rdfa import RDFaExtractor
from extruct.dublincore import DublinCoreExtractor
import extruct
jslde = JsonLdExtractor()
rdfa = RDFaExtractor()
ex = DublinCoreExtractor()



data = extruct.extract(soup.prettify())
pp.pprint(data)


{ 'dublincore': [ { 'elements': [ { 'URI': 'http://purl.org/dc/elements/1.1/description',
                                    'content': 'Смартфон Xiaomi 14: отзывы '
                                               'покупателей на Яндекс Маркете. '
                                               'Достоинства и недостатки '
                                               'товара. Важная информация о '
                                               'товаре Смартфон Xiaomi 14: '
                                               'описание, фотографии, цены, '
                                               'варианты доставки, магазины на '
                                               'карте.',
                                    'name': 'description'}],
                    'namespaces': {},
                    'terms': []}],
  'json-ld': [],
  'microdata': [ { 'properties': {'url': 'http://market.yandex.by'},
                   'type': 'https://schema.org/WebSite'},
                 { 'properti

In [27]:
data.keys()

dict_keys(['microdata'])

In [25]:
reviews_url = 'https://market.yandex.ru/product--xiaomi-14/1943226316/reviews'

In [27]:
import re
from bs4 import BeautifulSoup

def search_term_share(
    search_term: str,
    result_text: str
):
    len_search = len(search_term)
    len_result = len(result_text)
    start_pos = result_text.find(search_term)
    if start_pos != -1:
        return len_search / len_result
    else:
        return 0.0
def clear_str(s):
    return s.replace("Стоит ли покупать", "").replace("? Отзывы на Яндекс Маркете", "").strip().lower()
    
with open("duck_res.html") as fp:
    soup = BeautifulSoup(fp, "html.parser")

first_result = soup.find("article", id='r1-0')
second_result = soup.find("article", id='r1-1')

first_result_text = first_result.find("h2").find("span").text
second_result_text = second_result.find("h2").find("span").text

search_term = 'Xiaomi 14'.lower()



first_result_text = clear_str(first_result_text).lower()
second_result_text = clear_str(second_result_text).lower()

first_result_text.find(search_term)



first_share = search_term_share(search_term, first_result_text)
second_share = search_term_share(search_term, second_result_text)
if first_share >= second_share:
    chosen_result = first_result
else: 
    chosen_result = second_result

In [22]:
second_result_text

'смартфон xiaomi 14'

In [85]:
from fastbm25 import fastbm25

corpus = [
   first_result_text,
   second_result_text
]
tokenized_corpus = [doc.lower().split(" ") for doc in corpus]
model = fastbm25(tokenized_corpus)
query = search_term.lower().split()
result = model.top_k_sentence(query,k=1)
print(result)

[(['смартфон', 'xiaomi', '14', 'pro'], 0, -0.56)]


In [86]:
first_result_text

'Смартфон Xiaomi 14 Pro'

In [88]:
model.similarity_bm25(first_result_text, second_result_text)

0.0

In [89]:
query

['xiaomi', '14']

In [ ]:
link = top_result.find("a", {'href': re.compile(r'reviews')}).get('href')
print(link)

https://market.yandex.ru/product--smartfon-xiaomi-14-pro/1943141568/reviews


In [58]:
link.attrs

AttributeError: 'str' object has no attribute 'attrs'

In [53]:
link

'https://market.yandex.ru/product--xiaomi-14/1943226316/reviews'

In [ ]:
{'href': re.compile(r'crummy\.com/')}

In [30]:
len(top_result)

3

In [24]:
top_result.find("a",)

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [22]:
type(top_result)

bs4.element.ResultSet

In [26]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

import requests 
from bs4 import BeautifulSoup

url = 'https://yandex.by/search/'
params={
        'text':'Xiaomi 14',
        'lr':157,
        'search_source':'yaby_desktop_common',
        'src':'suggest_B',
    }
for i, (k,v ) in enumerate(params.items()):
    if i == 0:
        prefix = '?'
    else:
        prefix = '&'
        url += prefix + str(k) + '=' + str(v).replace(' ', '+')

from playwright.sync_api import sync_playwright
playwright = sync_playwright().start()
browser = playwright.firefox.launch(headless=True)
page = browser.new_page()
page.goto(reviews_url)
content = page.content()

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [ ]:
with open('google_res.html', 'w') as f:
    f.write(response.text)

In [ ]:
page.screenshot(path="example2.png")
browser.close()
playwright.stop()

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [34]:
url += '&' + str(k).replace(' ', '+') + '=' + str(v) 

In [39]:
url


'https://yandex.by/search/&text=Xiaomi+14&lr=157&search_source=yaby_desktop_common&src=suggest_B'

NameError: name 'page' is not defined

In [17]:
from selenium import webdriver

driver = webdriver.Chrome()  # can be webdriver.Chrome()
driver.get("about:blank")

data = '<h1>test</h1>'  # supposed to come from BeautifulSoup
driver.execute_script('document.body.innerHTML = "{html}";'.format(html=soup))

JavascriptException: Message: javascript error: Invalid or unexpected token
  (Session info: chrome=120.0.6099.71)
Stacktrace:
#0 0x564f9f274f83 <unknown>
#1 0x564f9ef2dcf7 <unknown>
#2 0x564f9ef342a3 <unknown>
#3 0x564f9ef36bb4 <unknown>
#4 0x564f9efc6ba3 <unknown>
#5 0x564f9efa70b2 <unknown>
#6 0x564f9efc6006 <unknown>
#7 0x564f9efa6e53 <unknown>
#8 0x564f9ef6edd4 <unknown>
#9 0x564f9ef701de <unknown>
#10 0x564f9f239531 <unknown>
#11 0x564f9f23d455 <unknown>
#12 0x564f9f225f55 <unknown>
#13 0x564f9f23e0ef <unknown>
#14 0x564f9f20999f <unknown>
#15 0x564f9f262008 <unknown>
#16 0x564f9f2621d7 <unknown>
#17 0x564f9f274124 <unknown>
#18 0x787605494ac3 <unknown>


In [22]:
!pip install selenium

  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 10.8 MB/s eta 0:00:00m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 9.4 MB/s eta 0:00:006 MB/s eta 0:00:01
Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [20]:
type(soup)

bs4.BeautifulSoup

In [ ]:
start=0
  &num=10
  &q=red+sox
  &cr=countryCA
  &lr=lang_fr
  &client=google-csbe
  &output=xml_no_dtd
  &cx=00255077836266642015:u-scht7a-8i

In [10]:

# response.text

In [13]:


headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }


def google(q):
    s = requests.Session()
    q = '+'.join(q.split())
    url = 'https://www.google.com/search?q=' + q + '&ie=utf-8&oe=utf-8'
    r = s.get(url, headers=headers_Get)

    soup = BeautifulSoup(r.text, "html.parser")
    output = []
    for searchWrapper in soup.find_all('h3', {'class':'r'}): #this line may change in future based on google's web page structure
        url = searchWrapper.find('a')["href"] 
        text = searchWrapper.find('a').text.strip()
        result = {'text': text, 'url': url}
        output.append(result)

    return output

In [14]:
output = google('Xiaomi 14')

In [15]:
output

[]

In [35]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

search = GoogleSearchAPIWrapper()

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

ValidationError: 1 validation error for GoogleSearchAPIWrapper
__root__
  Did not find google_api_key, please add an environment variable `GOOGLE_API_KEY` which contains it, or pass `google_api_key` as a named parameter. (type=value_error)

In [25]:
import requests

In [26]:
user_query = 'яндекс маркет отзывы Xiaomi 14'
query = f'https://www.google.com/search?q={user_query}'

In [27]:
results = requests.get(query)

In [20]:
search = DuckDuckGoSearchAPIWrapper(region="ru-ru", time="d", max_results=20, backend='html')
# searching_tool = DuckDuckGoSearchResults(api_wrapper=search,)

In [21]:
result = searching_tool.invoke('яндекс маркет отзывы')

In [22]:
search_run = DuckDuckGoSearchRun()

In [11]:
result = search_run.invoke('яндекс маркет отзывы')

In [14]:
search.invoke('яндекс маркет отзывы')

AttributeError: 'DuckDuckGoSearchAPIWrapper' object has no attribute 'invoke'

In [23]:
search.run(query='яндекс маркет отзывы Xiaomi 14')

'В DNS — 65 999 ₽ на «Яндекс Маркете» — от 50 959 ₽ ... тесты коллег-журналистов и отзывы пользователей, мы отобрали лучшие ноутбуки в бюджете до 70 000 ₽. ... Xiaomi Redmi Book Pro 14. Asus VivoBook 17 X1704ZA-AU121W. Изучаете отзывы про ТОП—7. Лучшие ноутбуки 13-14 дюймов. Рейтинг 2023 года!? Почитайте свежие реальные отзывы от имени бывших сотрудников на сайте shitcompany.org Стоит ли покупать Xiaomi Poco F5 Pro зимой 2023-2024 года? / Арстайл / - тема важная и интересная, поэтому наша редакция подготовила подробный разбор этой темы на сайте mnogorabotnikov.ru Лучшие смартфоны без ШИМ: Топ-5 смартфонов с IPS-экраном 📱 Рейтинг 2024 года - тема важная и интересная, поэтому наша редакция подготовила подробный разбор этой темы на сайте mnogorabotnikov.ru Изучаете отзывы про ТОП-6. Лучшие ноутбуки для работы и учебы ⚡ Рейтинг 2024 года по цене-качеству? Почитайте свежие реальные отзывы от имени бывших сотрудников на сайте shitcompany.org От 5 до 10 тысяч рублей выручают 16% респондентов,